In [1]:
import numpy as np
import pandas as pd

In [2]:
import numpy as np
import pandas as pd

def weighted_hs(rets, lam, window, p):
    """
    Compute the Weighted Historical Simulation Value-at-Risk (VaR).

    Parameters:
    rets (pd.Series): t-element pandas Series with a DatetimeIndex representing returns.
    lam (float): Smoothing parameter (0 < lam < 1).
    window (int): The size of the rolling window to compute the weighted CDF.
    p (float): The confidence level for VaR computation (e.g., 0.95 for 95% VaR).

    Returns:
    pd.Series: A series of 1-step ahead VaR forecasts with the first `window-1` values as NaN.
    """
    returns = pd.Series(index=rets.index, dtype=float)
    
    # Compute weights
    T = len(returns)
    # Initialize an array to store the weights
    weights = np.zeros(T)
    
    # Calculate the weight for T (the most recent period)
    weights[T-1] = (1 - lam) / (1 - lam ** window)
    
    # Calculate the weights for t < T using the recursive relationship
    for t in range(T-2, -1, -1):
        weights[t] = lam * weights[t+1]
    
    # need to check code below
    # calculate G (weighted empirical CDF)
    for t in range(window - 1, len(rets)):
        window_data = rets.iloc[t - window + 1 : t + 1].values  # Get the rolling window data
        sorted_indices = np.argsort(window_data)  # Sort returns
        sorted_returns = window_data[sorted_indices]
        sorted_weights = weights[sorted_indices].cumsum()  # Compute cumulative weights

        # Find the VaR as the quantile where cumulative probability reaches (1 - p)
        var_forecast.iloc[t] = sorted_returns[np.searchsorted(sorted_weights, 1 - p)]
    
    return var_forecast